<a href="https://colab.research.google.com/github/sakki221/Assingments/blob/main/Hackathon1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

train = pd.read_csv("hacktrain.csv")
test = pd.read_csv("hacktest.csv")

train = train.drop(columns=["Unnamed: 0"], errors='ignore')
test = test.drop(columns=["Unnamed: 0"], errors='ignore')

X_raw = train.drop(columns=["ID", "class"])
y = train["class"]
X_test_raw = test.drop(columns=["ID"])
test_ids = test["ID"]

def add_features(df):
    df_feat = df.copy()
    df_feat["NDVI_mean"] = df.mean(axis=1)
    df_feat["NDVI_std"] = df.std(axis=1)
    df_feat["NDVI_min"] = df.min(axis=1)
    df_feat["NDVI_max"] = df.max(axis=1)
    time = np.arange(df.shape[1])
    df_feat["NDVI_trend"] = df.apply(lambda row: np.polyfit(time, row.fillna(0), 1)[0], axis=1)
    return df_feat

X = add_features(X_raw)
X_test = add_features(X_test_raw)

le = LabelEncoder()
y_encoded = le.fit_transform(y)

pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(max_iter=1000, multi_class="multinomial", solver="lbfgs"))
])

X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

val_preds = pipeline.predict(X_val)
val_acc = accuracy_score(y_val, val_preds)
print(f"Validation Accuracy: {val_acc:.4f}")

test_preds = pipeline.predict(X_test)
test_preds_labels = le.inverse_transform(test_preds)

submission = pd.DataFrame({
    "ID": test_ids,
    "class": test_preds_labels
})
submission.to_csv("submission_logreg_features.csv", index=False)
print("✅ Submission saved as 'submission_logreg_features.csv'")


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Validation Accuracy: 0.9119
✅ Submission saved as 'submission_logreg_features.csv'
